In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from random import shuffle
from tqdm import tqdm
import os

# Display entire width of browser
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [ ]:
def extractImages(pathIn, pathOut):
    count = 0
    vidcap = cv.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    while success:
      vidcap.set(cv.CAP_PROP_POS_MSEC,(count*1000))    # added this line 
      success,image = vidcap.read()
      print('Read a new frame: ', success)     
      cv.imwrite( pathOut + "/frame%d.jpg" % count, image)     # save frame as JPEG file
      count = count + 1

In [ ]:
w = 96
h = 96

In [ ]:
def list_dir(path):
    fileNames = []
    for root, dirs, files in os.walk(path):  
        for filename in files: 
            fileNames.append(filename)            
    return fileNames

In [2]:
def invert(img):
    inverted = img[:,:,::-1]
    inverted[np.where((inverted > [127,127,127]).all(axis = 2))] = [0,0,0]
    return inverted

In [ ]:
files = list_dir('../videos')

In [ ]:
print(files[0])

In [ ]:
def extract(videoRoot, imageRoot):
    videos = list_dir(videoRoot)
    for video in videos:
        videoPath = videoRoot + '/' + video
        videoName = video.split('.')[0]
        imagePath = imageRoot+'/' +videoName
        if not os.path.exists(imagePath):
            os.makedirs(imagePath)
            extractImages(videoPath,imagePath)        

In [ ]:
extract('../videos', '../images')

In [ ]:
labelsFile = '../labels/labels.csv'
def read_labels(lablesFile):
    labelData = []
    with open(lablesFile) as inf:
        for line in tqdm(inf):            
            video, frame, label, clazz = line.strip().split(",")
            frame = int(frame)
            label = int(label)
            labelData.append({'video':video, 'frame':frame, 'label':label, 'class':clazz})
    return labelData

labels = read_labels(labelsFile)
            

In [ ]:
def get_label(file, labels):    
    for l in labels:         
        if 'frame' + str(l['frame']) + '.jpg' == file:
            if l['class'] == 'Light':                
                return np.array([1,0])
            else:
                return np.array([0,1])
    return np.array([1,1])
    
def train_data_with_label(train_data, labels):
    train_images = []    
    for i in tqdm(os.listdir(train_data)):        
        path = os.path.join(train_data, i)        
        img = cv.imread(path)
        img = invert(img)
        img = cv.resize(img, (w,h))     
        label = get_label(i, labels)      
        train_images.append([np.array(img),label])              
    #shuffle(train_images)
    return train_images

In [ ]:
train_data = train_data_with_label('../images/video-1', labels)

In [ ]:
train_data[100]

In [ ]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

In [ ]:
test_data = train_data_with_label('../images/video-1', labels)

In [ ]:
tr_img_data = np.array([i[0] for i in train_data]).reshape(-1, w,h,1)
tr_lbl_data = np.array([i[1] for i in train_data])

tst_img_data = np.array([i[0] for i in test_data]).reshape(-1, w,h,1)
tst_lbl_data = np.array([i[1] for i in test_data])


In [ ]:
for i in tr_lbl_data:
    print(i)

In [ ]:
model = Sequential()

model.add(InputLayer(input_shape=[w,h,1]))
model.add(Conv2D(filters=32, kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(2,activation='softmax'))
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=tr_img_data,y=tr_lbl_data,epochs=50,batch_size=10)
model.summary()

In [ ]:
    test_img = train_data[135][0]
    predict_data = test_img.reshape(-1,w,h,1)
    model_out = model.predict([predict_data])
    print(model_out)

In [ ]:
fig =plt.figure(figsize=(14,14))
plot_cols = 5
plot_rows = len(train_data)/plot_cols
for cnt, data in enumerate(train_data[:]):
    y = fig.add_subplot(plot_rows,plot_cols, cnt+1)
    img = data[0]
    data = img.reshape(1,w,h,1)
    model_out = model.predict([data])
    
    if np.argmax(model_out) == 1:
        str_label = 'No Lights'
    else:
        str_label = 'Lights'
        
    y.imshow(img, cmap='gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)    